In [ ]:
"""
Required Libraries for PyTorch Nueral network
Sets Torch Seed 
Sets Numpy Random Seed
And Random Seed for reproducibility
"""
import torch
#--- Seeding and Setting Deterministic GPU behavior ---#
torch.manual_seed(60794)
torch.backends.cudnn.deterministic = True
#--- Seeding for Numpy and Random Seeding ---#
import numpy as np
np.random.seed(60794)
import random
random.seed(60794)
#--- nueral network ---#
from torch import nn
from torch.nn import functional as F
from torch.utils import data
#--- Additional Libraries ---#
import os
import glob
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torchvision
from pathlib import Path
from tqdm import tqdm, tqdm_notebook
from torchvision import models
from sklearn.metrics import jaccard_similarity_score
from torch.autograd import Variable
import _pickle
#----- Library Code END -----#

In [ ]:
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels, y_pred = y_true_in, y_pred_in
    true_objects, pred_objects = 2, 2
    # Jiaxin fin that if all zeros, then, the background is treated as object
    temp1 = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=([0,0.5,1], [0,0.5, 1]))
    intersection = temp1[0]
    
    # Compute areas (needed for finding the union between all objects)
    #print(np.histogram(labels, bins = true_objects))
    area_true = np.histogram(labels,bins=[0,0.5,1])[0]
    #print("area_true = ",area_true)
    area_pred = np.histogram(y_pred, bins=[0,0.5,1])[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    
    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    intersection[intersection == 0] = 1e-9
    
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    y_pred_in = y_pred_in > 0.5 # added by sgx 20180728
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)


In [ ]:
import math
import operator
import copy


def set_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def get_learning_rate(optimizer):
    return optimizer.param_groups[0]['lr']



class LearningRate():
    def __init__(self, initial_lr, iteration_type):
        self.initial_lr = initial_lr
        self.iteration_type = iteration_type #epoch or mini_batch

    def get_learning_rate(self, optimizer):
        return optimizer.param_groups[0]['lr']

    def set_learning_rate(self, optimizer, new_lr):
        for param_group in optimizer.param_groups:
            param_group['lr'] = new_lr

    def adjust(self, optimizer, lr, iteration, params=None):
        self.set_learning_rate(optimizer, lr)
        return lr


class FixedLR(LearningRate):
    def __init__(self, initial_lr, iteration_type):
        super().__init__(initial_lr, iteration_type)

    def adjust(self, optimizer, iteration, params=None):
        new_lr = super().get_learning_rate(optimizer)
        return new_lr


class LinearLR(LearningRate):
    def __init__(self, initial_lr, iteration_type, fixed_delta):
        super().__init__(initial_lr, iteration_type)
        self.fixed_delta = fixed_delta

    def adjust(self, optimizer, iteration, params=None):
        lr = super().get_learning_rate(optimizer)
        new_lr = lr + self.fixed_delta
        super().set_learning_rate(optimizer, new_lr)
        return new_lr


class SnapshotLR(LearningRate):
    '''https://arxiv.org/abs/1704.00109'''
    def __init__(self, initial_lr, iteration_type,
                 max_lr, total_iters, n_cycles):
        '''
        n_iters = total number of mini-batch iterations during training
        n_cycles = total num snapshots during training
        max_lr = starting learning rate each cycle'''
        super().__init__(initial_lr, iteration_type)
        self.max_lr = max_lr
        self.total_iters = total_iters
        self.cycles = n_cycles

    def cosine_annealing(self, t):
        '''t = current mini-batch iteration'''
        return self.max_lr/2 * (math.cos(
         (math.pi * (t % (self.total_iters//self.cycles))) /
         (self.total_iters//self.cycles)) + 1)

    def adjust(self, optimizer, iteration, params=None):
        new_lr = self.cosine_annealing(iteration)
        self.set_learning_rate(optimizer, new_lr)
        return new_lr


class SnapshotParamsLR(LearningRate):
    '''Snapshot Learning with per-parameter LRs'''
    def __init__(self, initial_lr, iteration_type,
                 total_iters, n_cycles):
        '''
        n_iters = total number of mini-batch iterations during training
        n_cycles = total num snapshots during training
        max_lr = starting learning rate each cycle'''
        super().__init__(initial_lr, iteration_type)
        self.total_iters = total_iters
        self.cycles = n_cycles

    def cosine_annealing(self, t, max_lr):
        return max_lr/2 * (math.cos(
         (math.pi * (t % (self.total_iters//self.cycles)))/(
            self.total_iters//self.cycles)) + 1)

    def adjust(self, optimizer, iteration, params=None):
        lrs = []
        for param_group in optimizer.param_groups:
            new_lr = self.cosine_annealing(iteration, param_group['max_lr'])
            param_group['lr'] = new_lr
            lrs.append(new_lr)
        return new_lr


class DevDecayLR(LearningRate):
    '''https://arxiv.org/abs/1705.08292'''
    def __init__(self, initial_lr, iteration_type,
                 decay_factor=0.9, decay_patience=1):
        super().__init__(initial_lr, iteration_type)
        self.decay_factor = decay_factor
        self.decay_patience = decay_patience

    def adjust(self, optimizer, iteration, params):
        lr = super().get_learning_rate(optimizer)
        best_iter = params['best_iter']

        if (iteration - best_iter) > self.decay_patience:
            print('Decaying learning rate by factor: {:.5f}'.format(
                self.decay_factor).rstrip('0'))
            lr *= self.decay_factor
            super().set_learning_rate(optimizer, lr)
        return lr


class ScheduledLR(LearningRate):
    def __init__(self, initial_lr, iteration_type, lr_schedule):
        super().__init__(initial_lr, iteration_type)
        self.lr_schedule = lr_schedule

    def adjust(self, optimizer, iteration, params=None):
        if iteration in self.lr_schedule:
            new_lr = self.lr_schedule[iteration]
        else:
            new_lr = self.get_learning_rate(optimizer)
        super().set_learning_rate(optimizer, new_lr)
        return new_lr


class DecayingLR(LearningRate):
    def __init__(self, initial_lr, iteration_type, decay, n_epochs):
         super().__init__(initial_lr, iteration_type)
         self.decay = decay
         self.n_epochs = n_epochs

    def exponential_decay(self, iteration, params=None):
        '''Update learning rate to `initial_lr` decayed
        by `decay` every `n_epochs`'''
        return self.initial_lr * (self.decay ** (iteration // self.n_epochs))

    def adjust(self, optimizer, iteration):
        new_lr = self.exponential_decay(iteration)
        super().set_learning_rate(optimizer, new_lr)
        return new_lr


class CyclicalLR(LearningRate):
    '''https://arxiv.org/abs/1506.01186'''
    def __init__(self, initial_lr, iteration_type, n_iters, cycle_length,
                 min_lr, max_lr):
         assert initial_lr == min_lr
         super(CyclicalLR, self).__init__(initial_lr, iteration_type)
         self.n_iters = n_iters
         self.cycle_length = cycle_length
         self.min_lr = min_lr
         self.max_lr = max_lr

    def triangular(self, iteration):
        iteration -= 1 # if iteration count starts at 1
        cycle = math.floor(1 + iteration/self.cycle_length)
        x = abs(iteration/(self.cycle_length/2) - 2*cycle + 1)
        new_lr = self.min_lr + (self.max_lr - self.min_lr) * max(0, (1-x))
        return new_lr

    def adjust(self, optimizer, iteration, best_iter=1):
        new_lr = self.triangular(iteration)
        super().set_learning_rate(optimizer, new_lr)
        return new_lr




## Helpers

def cosine_annealing(lr_max, T, M, t):
    '''
    t = current mini-batch iteration
    # lr(t) = f(t-1 % T//M)
    # lr(t) = lr_max/2 * (math.cos( (math.pi * (t % T/M))/(T/M) ) + 1)
    '''
    return lr_max/2 * (math.cos( (math.pi * (t % (T//M)))/(T//M)) + 1)

In [ ]:
"""
Keras -> Pytorch considerations
padding 0 is "valid"
padding 1 is "same"
"""
def conv3by3(in_ch,out_ch,padding):
	"""
	Conv2d
	(3 x 3) kernel size
	in_ch input_channels
	out_ch output_channels
	padding - 0: equivalent to "valid"
	          1: equivalent to "same" 
	"""
	return nn.Conv2d(in_ch,out_ch,3,padding=padding)

def conv1by1(in_ch,out_ch,padding):
	"""
	Conv2d
	(1 x 1) kernel size
	in_ch input_channels
	out_ch output_channels
	padding - 0: equivalent to "valid"
	          1: equivalent to "same" 
	"""
	return nn.Conv2d(in_ch,out_ch,1,padding=padding)

def convtrans3by3(in_ch,out_ch,padding):
	"""
	Convtrans
	Stride = 2
	(3 x 3) kernel size
	in_ch input_channels
	out_ch output_channels
	padding - 0: equivalent to "valid"
	          1: equivalent to "same" 
	"""
	return nn.ConvTranspose2d(in_ch,out_ch,3,stride=2,padding=padding,output_padding=padding)

def swish(x):
	"""
	Swish Activation function implementation
	Added for improvement on ReLU activation
	Link: https://arxiv.org/abs/1710.05941
	Cite: arXiv:1710.05941v2 [cs.NE]
	"""
	return torch.sigmoid(x) * x

class GlobalAvgPool(nn.Module):
	def __init__(self):
		super(GlobalAvgPool,self).__init__()
	def forward(self, x):
		return x.view(*(x.shape[:-2]),-1).mean(-1)

class Seq_Ex_Block(nn.Module):
	"""
	Squeeze - Excite Block
	Source Kaggle Discussion Forum
	__insert article link____
	"""
	def __init__(self,in_ch,r=16):
		super(Seq_Ex_Block,self).__init__()
		self.se = nn.Sequential(
			GlobalAvgPool(),
			nn.Linear(in_ch, in_ch//r),
			nn.ReLU(inplace=True),
			nn.Linear(in_ch//r, in_ch),
			nn.Sigmoid()
			)
	def forward(self,x):
		se_weight = self.se(x).unsqueeze(-1).unsqueeze(-1)
		return x.mul(se_weight)

class conv_block(nn.Module):
	"""
	Convolutional Block
	Activation can be changed here
	Conv2d -> BatchNorm2d (-> Activation)
	"""
	def __init__(self, in_ch, out_ch,activation):
		super().__init__()
		self.conv = conv3by3(in_ch,out_ch,1)
		self.batch_norm = nn.BatchNorm2d(out_ch)
		self.instance_norm = nn.InstanceNorm2d(out_ch)
		#self.layer_norm = nn.LayerNorm(out_ch)
		self.activation = activation
		self.activ = nn.LeakyReLU(0.001,inplace=True)
# added depth wise convolutions here
	def forward(self,x):
		if self.activation == True:
			x = self.activ(self.instance_norm(self.batch_norm(self.conv(x))))
			#x = swish(self.layer_norm(self.conv(x)))
		else:
			x = self.instance_norm(self.batch_norm(self.conv(x)))
			#x = self.layer_norm(self.conv(x))
		return x

class res_block(nn.Module):
	"""
	Residual Block
	input
	x = input -> conv block w activation -> conv block w/o activation
	element wise addition x + input
	Squeeze and Excite Block can be added here
	"""
	def __init__(self, in_ch, out_ch):
		super().__init__()
		self.activation = nn.ReLU(inplace=True)
		self.batch_norm = nn.BatchNorm2d(in_ch)
		self.conv_block_activated = conv_block(in_ch,out_ch,True)
		self.conv_block_no_activ = conv_block(in_ch,out_ch,False)
		self.squeeze = Seq_Ex_Block(in_ch)

	def forward(self,x):
		blockInput = x
		x = self.batch_norm(self.activation(x))
		x = self.conv_block_activated(x)
		x = self.conv_block_no_activ(x)
		x = self.squeeze(x)
		x = torch.add(x,blockInput)
		return x

# This model is not modular b/c all values are set
class UNet(nn.Module):
	"""
	WARNING!!! WARNING!!!WARNING!!! WARNING!!!WARNING!!!
	---------------------------------------------------
	--- Not entirely modular until calculations are ---
	--- not built on a start_nuerons argument. --------
	---------------------------------------------------
	WARNING!!! WARNING!!!WARNING!!! WARNING!!!WARNING!!!

	Model Architecture
	Composed of Conv and Residual Blocks
	Where we would add CoordinateChannel,
	DepthChannel. Tuning on dropout // optimal
	on 0.6 through arduous Keras testing
	"""
	def __init__(self,dropout_ratio=0.6):
		super().__init__()
		# -- Recurring network modules -- #
		self.relu = nn.ReLU(inplace=True)
		self.sigmoid = nn.Sigmoid()
		self.pool = nn.MaxPool2d(2)
		self.dropoutA = nn.Dropout2d(p = dropout_ratio/4)
		self.dropoutB = nn.Dropout2d(p = dropout_ratio/2)

		# 101 - 50
		self.conv1 = conv3by3(3,16,1)
		self.r_b1 = res_block(16,16)
		self.r_b2 = res_block(16,16)
		# 50 - 25
		self.conv2 = conv3by3(16,32,1)
		self.r_b3 = res_block(32,32)
		self.r_b4 = res_block(32,32)
		# 25 - 12
		self.conv3 = conv3by3(32,64,1)
		self.r_b5 = res_block(64,64)
		self.r_b6 = res_block(64,64)
		# 12 - 6
		self.conv4 = conv3by3(64,128,1)
		self.r_b7 = res_block(128,128)
		self.r_b8 = res_block(128,128)
		# Middle
		self.convm = conv3by3(128,256,1)
		self.r_bm1 = res_block(256,256)
		self.r_bm2 = res_block(256,256)
		# 6 - 12
		self.deconv4 = convtrans3by3(256,128,1)
		self.uconv4 = conv3by3(256,128,1)
		self.r_bu1 = res_block(128,128)
		self.r_bu2 = res_block(128,128)
		# 12 - 25
		self.deconv3 = convtrans3by3(128,64,1)
		self.uconv3 = conv3by3(128,64,1)
		self.r_bu3 = res_block(64,64)
		self.r_bu4 = res_block(64,64)
		# 25 - 50
		self.deconv2 = convtrans3by3(64,32,1)
		self.uconv2 = conv3by3(64,32,1)
		self.r_bu5 = res_block(32,32)
		self.r_bu6 = res_block(32,32)
		# 50 - 101
		self.deconv1 = convtrans3by3(32,16,1)
		self.uconv1 = conv3by3(32,16,1)
		self.r_bu7 = res_block(16,16)
		self.r_bu8 = res_block(16,16)
		# Final Part
		self.final_conv = conv1by1(16,1,0)

	def forward(self,x):
		#print(x.shape)
		# 101 -> 50
		conv_1 = swish(self.r_b2(self.r_b1(self.conv1(x))))
		pool_1 = self.dropoutA(self.pool(conv_1))
		#print("Conv1: ",conv_1.shape)
		#print("Pool1: ",pool_1.shape)
		# 50 -> 25
		conv_2 = swish(self.r_b4(self.r_b3(self.conv2(pool_1))))
		pool_2 = self.dropoutA(self.pool(conv_2))
		#print("Conv2: ",conv_2.shape)
		#print("Pool2: ",pool_2.shape)
		# 25 -> 12
		conv_3 = swish(self.r_b6(self.r_b5(self.conv3(pool_2))))
		pool_3 = self.dropoutA(self.pool(conv_3))
		#print("Conv3: ",conv_3.shape)
		#print("Pool3: ",pool_3.shape)
		# 12 -> 6
		conv_4 = swish(self.r_b8(self.r_b7(self.conv4(pool_3))))
		pool_4 = self.dropoutA(self.pool(conv_4))
		#print("Conv4: ",conv_4.shape)
		#print("Pool4: ",pool_4.shape)
		# Middle
		conv_m = swish(self.r_bm2(self.r_bm1(self.convm(pool_4))))
		#print("ConvM: ",conv_m.shape)
		# 6 -> 12
		de_conv_4 = self.deconv4(conv_m)
		uconv_4 = torch.cat([de_conv_4,conv_4],1)
		#print("Uconv4: ",uconv_4.shape)
		uconv_4 = self.dropoutB(uconv_4)
		uconv_4 = self.uconv4(uconv_4)
		uconv_4 = self.r_bu1(uconv_4)
		uconv_4 = swish(self.r_bu2(uconv_4))
		#print("Uconv4: ",uconv_4.shape)
		# 12 -> 25
		de_conv_3 = self.deconv3(uconv_4)
		uconv_3 = torch.cat([de_conv_3,conv_3],1)
		uconv_3 = self.dropoutB(uconv_3)
		uconv_3 = self.uconv3(uconv_3)
		uconv_3 = self.r_bu3(uconv_3)
		uconv_3 = swish(self.r_bu4(uconv_3))
		#print("Uconv3:",uconv_3.shape)
		# 25 -> 50
		de_conv_2 = self.deconv2(uconv_3)
		uconv_2 = torch.cat([de_conv_2,conv_2],1)
		uconv_2 = self.dropoutB(uconv_2)
		uconv_2 = self.uconv2(uconv_2)
		uconv_2 = self.r_bu5(uconv_2)
		uconv_2 = swish(self.r_bu6(uconv_2))
		#print("Uconv2:",uconv_2.shape)
		#50 -> 101
		de_conv_1 = self.deconv1(uconv_2)
		uconv_1 = torch.cat([de_conv_1,conv_1],1)
		uconv_1 = self.dropoutB(uconv_1)
		uconv_1 = self.uconv1(uconv_1)
		uconv_1 = self.r_bu7(uconv_1)
		uconv_1 = swish(self.r_bu8(uconv_1))
		#print("Uconv1:",uconv_1.shape)
		#Final
		conv_final = self.final_conv(uconv_1)
		conv_final = self.sigmoid(conv_final)
		#print("Conv_final: ",conv_final.shape)
		return conv_final

def gael_net(**kwargs):
    model = UNet(**kwargs)
    return model

def get_model():
    model = gael_net()
    model.train()
    return model.cuda()


In [ ]:
# https://github.com/leigh-plt/cs231n_hw2018/blob/master/assignment2/pytorch_tutorial.ipynb
"""
Save and Load Checkpoints
Based on Model Training
Source: https://github.com/leigh-plt/cs231n_hw2018/blob/master/assignment2/pytorch_tutorial.ipynb
Will be the basis for checkpoint ensembling
"""
def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [ ]:
def train_model(model,epochs,
                learning_rate,loss_function,
                optimizer, dataset, dataset_val,
                batch_size
               ):
    """
    function for training our model
    """
    snapshot = SnapshotLR(
        initial_lr=0.000001,max_lr=0.0001,
        total_iters=epochs,n_cycles=30,
        iteration_type="epochs"
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                           mode="max",
                                                           factor=0.97,
                                                           min_lr=0,
                                                           patience=10
                                                          )
    lowest_train_loss = None
    lowest_val_loss = None
    highest_train_iou = None
    highest_val_iou = None
    for epoch in range(epochs):
        # TRAIN BATCH
        train_loss = []
        train_iou = []
        for image, mask in tqdm_notebook(data.DataLoader(dataset,batch_size=batch_size,shuffle=True)):
            image = image.type(torch.FloatTensor).cuda()
            y_pred = model(Variable(image))
            loss = loss_function(y_pred, Variable(mask.cuda()))
            iou_train = iou_metric_batch(mask.cpu().numpy(),y_pred.cpu().detach().numpy())
            
            optimizer.zero_grad()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
            optimizer.step()
            train_loss.append(loss.data[0])
            train_iou.append(iou_train)
        
        # VAL BATCH
        val_loss = []
        val_iou = []
        for image, mask in data.DataLoader(dataset_val, batch_size = batch_size, shuffle = False):
            image = image.cuda()
            y_pred = model(Variable(image))
            
            loss = loss_function(y_pred, Variable(mask.cuda()))
            val_loss.append(loss.data[0])
            iou_val = iou_metric_batch(mask.cpu().numpy(),y_pred.cpu().detach().numpy())
            val_iou.append(iou_val)
        
        if lowest_train_loss is None:
            lowest_train_loss = np.mean(train_loss)
            save_checkpoint('low_train_loss_tgs.pth' ,model, optimizer)
        if lowest_val_loss is None:
            lowest_val_loss = np.mean(val_loss)
            save_checkpoint('low_val_loss_tgs.pth' ,model, optimizer)
        if highest_train_iou is None:
            highest_train_iou = np.mean(train_iou)
            save_checkpoint('high_train_iou_tgs.pth' ,model, optimizer)
        if highest_val_iou is None:
            highest_val_iou = np.mean(val_iou)
            save_checkpoint('high_val_iou_tgs.pth' ,model, optimizer)
        print("Epoch: %d,T_Loss: %.7f, T_IOU: %.7f, V_Loss: %.7f, V_IOU: %.7f" % (
            epoch,
            np.mean(train_loss),
            np.mean(train_iou),
            np.mean(val_loss),
            np.mean(val_iou)
        )
             )
        if np.mean(train_loss) < lowest_train_loss:
            lowest_train_loss = np.mean(train_loss)
            save_checkpoint('low_train_loss_tgs.pth' ,model, optimizer)
        if np.mean(val_loss) < lowest_val_loss:
            lowest_val_loss = np.mean(val_loss)
            save_checkpoint('low_val_loss_tgs.pth' ,model, optimizer)
        if np.mean(train_iou) > highest_train_iou:
            highest_train_iou = np.mean(train_iou)
            save_checkpoint('high_train_iou_tgs.pth' ,model, optimizer)
        if np.mean(val_iou) > highest_val_iou:
            highest_val_iou = np.mean(val_iou)
            save_checkpoint('high_val_iou_tgs.pth' ,model, optimizer)
        scheduler.step(highest_val_iou)
        resulting_lr = snapshot.adjust(optimizer,epoch)
        print("LR",resulting_lr)
    # Saves model post all epochs
    save_checkpoint('tgs-%i.pth' % epochs, model, optimizer)
    print("Best T_L:",lowest_train_loss,
          " Best T_I:",highest_train_iou,
          " Best V_L:",lowest_val_loss,
          " Best V_I:", highest_val_iou
         )
    return model

In [ ]:
class TGSSaltDataset(data.Dataset):
    def __init__(self, root_path, file_list, is_test = False):
        self.is_test = is_test
        self.root_path = root_path
        self.file_list = file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        if index not in range(0, len(self.file_list)):
            return self.__getitem__(np.random.randint(0, self.__len__()))
        
        file_id = self.file_list[index]
        
        image_folder = os.path.join(self.root_path, "images")
        image_path = os.path.join(image_folder, file_id + ".png")
        
        mask_folder = os.path.join(self.root_path, "masks")
        mask_path = os.path.join(mask_folder, file_id + ".png")
        
        image = load_image(image_path)
        
        if self.is_test:
            return (image,)
        else:
            mask = load_image(mask_path, mask = True)
            return image, mask

In [ ]:
def rle_encoding(x):
    """
    RLE Encoding function
    For finding the coordinates from 
    the predicted mask
    """
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

In [ ]:
def load_image(path,pad=True, mask = False):
    """
    Loads an image or its given mask.
    If (pad== True):
     Pads it so that it is
    easier to perform maxpooling and deconvolutions
    Else:
     Performs no padding and leaves image as is
    If (mask == True):
     perform mask specific conversion
    Else:
     does normal image conversion
    """
    flip = False
    if "_gael" in path:
        path = path.replace("_gael","")
        flip = True
    img = cv2.imread(str(path))
    if flip:
        #horizontal flip aka fliplr by cv2
        img = cv2.flip( img, 0 )
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, _ = img.shape

    if pad:
        if height % 32 == 0:
            y_min_pad = 0
            y_max_pad = 0
        else:
            y_pad = 32 - height % 32
            y_min_pad = int(y_pad / 2)
            y_max_pad = y_pad - y_min_pad
        
        if width % 32 == 0:
            x_min_pad = 0
            x_max_pad = 0
        else:
            x_pad = 32 - width % 32
            x_min_pad = int(x_pad / 2)
            x_max_pad = x_pad - x_min_pad
    
    img = cv2.copyMakeBorder(img, y_min_pad, y_max_pad, x_min_pad, x_max_pad, cv2.BORDER_REFLECT_101)
    
    if mask:
        img = img[:, :, 0:1] // 255
        return torch.from_numpy(np.transpose(img, (2, 0, 1)).astype('float32'))
    else:
        img = img / 255.0
        return torch.from_numpy(np.transpose(img, (2, 0, 1)).astype('float32'))


In [ ]:
#all_predictions_stacked = pred_resize(all_predictions_stacked,101,101)
def pred_resize(predictions,height,width):
    """
    Just resizes image to desired size
    """
    if height % 32 == 0:
        y_min_pad = 0
        y_max_pad = 0
    else:
        y_pad = 32 - height % 32
        y_min_pad = int(y_pad / 2)
        y_max_pad = y_pad - y_min_pad
    if width % 32 == 0:
        x_min_pad = 0
        x_max_pad = 0
    else:
        x_pad = 32 - width % 32
        x_min_pad = int(x_pad / 2)
        x_max_pad = x_pad - x_min_pad
    predictions = predictions[:, y_min_pad:128 - y_max_pad, x_min_pad:128 - x_max_pad]
    print("Prediction Shape: ",predictions.shape)
    return predictions

In [ ]:
def stack_and_resize(predictions,height,width):
    """
    Stacks and resizes the predicitons
    """
    preds_stacked = np.vstack(predictions)[:, 0, :, :]
    return pred_resize(preds_stacked,height,width)   

In [ ]:
# Silence All Warnings
warnings.filterwarnings("ignore")

In [ ]:
"""
Data Preparation and Gathering
1. Load in training and validation data
---Image and it's mask-----
2. Load in testing data
A - Pad images where necessary
Where would do analysis and update of data
"""
# Directory which contains our data
directory = '../input/tgs-salt-identification-challenge'

# Gets our train_data
depths_df = pd.read_csv(os.path.join(directory, 'train.csv'))

# Setting paths for our datasets
train_path = os.path.join(directory, 'train')
test_path = os.path.join(directory, 'test')

#Custom Test Split based on salt coverage
ids_val = _pickle.load(open("../input/intermediatetgs/val_index.obj","rb"))
ids_train = _pickle.load(open("../input/intermediatetgs/train_index.obj","rb"))

# Setting the file list
file_list = list(depths_df['id'].values)
#file_list_val = file_list[::10]
file_list_val = list(ids_val)
#file_list_train = [f for f in file_list if f not in file_list_val]
file_list_train = list(ids_train)
file_list_flip = [s + "_gael" for s in file_list_train]
file_list_train = file_list_train + file_list_flip
test_file_list = glob.glob(os.path.join(test_path, 'images', '*.png'))
test_file_list = [f.split('/')[-1].split('.')[0] for f in test_file_list]

# Validating our file list
print("Training Data:")
print("Train: ",len(file_list_train))
print("Validation: ",len(file_list_val))
print("Total: ",len(file_list))
print("Testing Data:")
print("Images: ",len(test_file_list))

# Datasets
dataset = TGSSaltDataset(train_path, file_list_train)
dataset_val = TGSSaltDataset(train_path, file_list_val)
test_dataset = TGSSaltDataset(test_path, test_file_list, is_test = True)

In [ ]:
"""
Model Training
Set epochs, learning rate,
loss functions, optimizers,
batch_size, shuffle
Checkpoint Saving
Also what is reported on every epoch
"""
model = get_model()


In [ ]:
#epoch =115
#learning_rate = 0.001
#loss_fn = torch.nn.BCELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, eps=1e-7)
#model = train_model(model,epoch,learning_rate,
#                    loss_fn,optimizer, dataset,
#                    dataset_val,32
#                   )

In [ ]:
#Load From Checkpoint
epoch =100
learning_rate = 0.0001
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, eps=1e-7)
load_checkpoint("../input/80pytorchgamma/high_val_iou_tgs.pth", model, optimizer)
model = train_model(model,epoch,learning_rate,
                    loss_fn,optimizer, dataset,
                    dataset_val,32
                   )

In [ ]:
load_checkpoint("high_val_iou_tgs.pth", model, optimizer)
# Making predictions Batch Size is the same
all_predictions = []
for image in tqdm_notebook(data.DataLoader(test_dataset, batch_size = 32)):
    image = image[0].type(torch.FloatTensor).cuda()
    y_pred = model(Variable(image)).cpu().data.numpy()
    all_predictions.append(y_pred)
all_predictions_stacked = stack_and_resize(all_predictions,101,101)

In [ ]:
# Making predictions of the validation sets
# and masks to determine iou
val_predictions = []
val_masks = []
for image, mask in tqdm_notebook(data.DataLoader(dataset_val, batch_size = 32)):
    image = Variable(image.type(torch.FloatTensor).cuda())
    y_pred = model(image).cpu().data.numpy()
    val_predictions.append(y_pred)
    val_masks.append(mask)
    

val_predictions_stacked = stack_and_resize(val_predictions,101,101)
val_masks_stacked = stack_and_resize(val_masks,101,101)

In [ ]:
# IOU Metric By Threshold
metric_by_threshold = []
for threshold in np.linspace(0.3, 0.7, 50):
    val_binary_prediction = (val_predictions_stacked > threshold).astype(int)
    
    iou_values = []
    for y_mask, p_mask in zip(val_masks_stacked, val_binary_prediction):
        iou = jaccard_similarity_score(y_mask.flatten(), p_mask.flatten())
        iou_values.append(iou)
    iou_values = np.array(iou_values)
    
    accuracies = [
        np.mean(iou_values > iou_threshold)
        for iou_threshold in np.linspace(0.5, 0.95, 10)
    ]
    print('Threshold: %.2f, Metric: %.3f' % (threshold, np.mean(accuracies)))
    metric_by_threshold.append((np.mean(accuracies), threshold))
    
best_metric, best_threshold = max(metric_by_threshold)

In [ ]:
threshold = best_threshold
binary_prediction = (all_predictions_stacked > threshold).astype(int)

all_masks = []
for p_mask in list(binary_prediction):
    p_mask = rle_encoding(p_mask)
    all_masks.append(' '.join(map(str, p_mask)))

In [ ]:
# Submission File
submit = pd.DataFrame([test_file_list, all_masks]).T
submit.columns = ['id', 'rle_mask']
submit.to_csv('submit_baseline_torch.csv', index = False)